In [1]:
import pyarrow as pa
import pyarrow.dataset as ds
import pandas as pd
from datasets import Dataset
from datasets import Image as Image2
import json
import os
import numpy as np
from PIL import Image, ImageDraw
from transformers import AutoModelForObjectDetection
from transformers import AutoImageProcessor
from datasets import load_dataset
from functools import partial
import albumentations
import numpy as np
import torch
from transformers import TrainingArguments
from transformers import Trainer

In [2]:
example_dir = 'examples/'
example_imges = os.listdir(example_dir)
def extractCoords(width, height, box, add_size=2):
    x, y, x2, y2 = tuple(box)
    xs = np.sort([x, x2])
    ys = np.sort([y, y2])
    x = np.floor(xs[0])-add_size
    x2 = np.ceil(xs[1])+add_size
    y = np.floor(ys[0])-add_size
    y2 = np.ceil(ys[1])+add_size
    if x < 0:
        x = 0
    if y<0:
        y = 0
    if x2>width:
        x2 = width
    if y2>height:
        y2 = height
    return (x, y, x2, y2)

In [3]:
mdl_name = 'detr-resnet-50_finetuned_pkr/checkpoint-10648/'
image_processor = AutoImageProcessor.from_pretrained(mdl_name)
model = AutoModelForObjectDetection.from_pretrained(mdl_name)

Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.48, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.


In [4]:
example_out_dir = 'out/'

In [5]:
img_out_ind = 0
for ii in range(0, len(example_imges)):
    fn = f'{example_dir}{example_imges[ii]}'
    print(f'Processing {fn}')
    image = Image.open(fn)
    
    fix = Image.new("RGB", image.size, (255, 255, 255)) # remove transparency if needed
    fix.paste(image) # 3 is the alpha channel
    image = fix
    width, height = image.size
    with torch.no_grad():
        inputs = image_processor(images=image, return_tensors="pt")
        outputs = model(**inputs)
        target_sizes = torch.tensor([image.size[::-1]])
        results = image_processor.post_process_object_detection(outputs, threshold=0.9, target_sizes=target_sizes)[0]
    
    for score, label, box in zip(results["scores"], results["labels"], results["boxes"]):
        box = [round(i, 2) for i in box.tolist()]
        print(
            f"Detected {model.config.id2label[label.item()]} with confidence "
            f"{round(score.item(), 3)} at location {box}"
        )
    draw = ImageDraw.Draw(image)
    
    for score, label, box in zip(results["scores"], results["labels"], results["boxes"]):
        box = [round(i, 2) for i in box.tolist()]
        x, y, x2, y2 = tuple(box)
        crop_coords = extractCoords(width, height, box)
        c_range = image.crop(crop_coords).save(f'{example_out_dir}{img_out_ind}.png')
        img_out_ind += 1

Processing examples/im_1.JPEG
Detected pk_range with confidence 0.999 at location [40.93, 52.5, 229.22, 241.54]
Processing examples/im_9.png
Detected pk_range with confidence 0.999 at location [85.94, 293.17, 820.85, 744.1]
Processing examples/im_7.PNG
Detected pk_range with confidence 1.0 at location [7.74, 48.08, 410.35, 455.0]
Processing examples/im_6.JPEG
Detected pk_range with confidence 1.0 at location [0.32, 1.26, 407.2, 407.31]
Processing examples/im_10.png
Detected pk_range with confidence 0.999 at location [101.55, 294.03, 839.68, 743.97]
Processing examples/im_0.JPEG
Detected pk_range with confidence 1.0 at location [103.32, 5.91, 371.01, 261.99]
Processing examples/im_5.JPEG
Detected pk_range with confidence 0.999 at location [31.03, 574.96, 356.76, 898.55]
Detected pk_range with confidence 0.998 at location [992.68, 575.91, 1299.91, 887.03]
Detected pk_range with confidence 0.996 at location [378.41, 134.63, 683.32, 446.41]
Detected pk_range with confidence 0.998 at locati